### Jul AdventKalender D20

https://adventofcode.com/2023/day/20

#### Part 1 

In [1]:
def construct_modules(path = "data/input20_test1.txt"):
    modules = {}
    modules['button']={'type':'button', 'next': ['broadcaster']}
    with open(path,"r") as file:
        for line in file.readlines():
            line = line.strip()
            key, values = line.split(' -> ')
            connected_modules = values.split(', ')
            if key.startswith('%'):
                modules[key[1:]] = {'type':'flip-flop', 'next': connected_modules, 'switch': 0} # switch: current status, 0 - 'off', 1 - 'on'
            elif key.startswith('&'):
                modules[key[1:]] = {'type':'conjunction', 'next': connected_modules, 'pulses':{}} # pulses: default all low
            else: # broadcaster
                modules[key] = {'type':'broadcast', 'next': connected_modules}
    return modules

def parse_conjunctions_pulses(modules):
    con_names = []
    connected = {}
    for key, value in modules.items():
        if value['type'] == 'conjunction':
            con_names.append(key)
            connected[key] = set()
    for key, value in modules.items():
        if value['type'] != 'conjunction':
            for con in con_names:
                if con in value['next']:
                    connected[con].add(key)
    for key, value in modules.items():
        if value['type'] == 'conjunction':
            modules[key]['pulses'] = { ck: 'low' for ck in connected[key]}
    return modules

In [2]:
def module_process(from_module_name, module, pulse):
    if module['type'] == 'button':
        return 'low', module['next']
    if module['type'] == 'broadcast':
        return pulse, module['next']
    if module['type'] == 'flip-flop':
        if pulse == 'high':
            return None, None
        if module['switch'] == 0:
            module['switch'] = 1
            return 'high', module['next']
        module['switch'] = 0
        return 'low', module['next']
    if module['type'] == 'conjunction':
        if from_module_name is not None:
            module['pulses'][from_module_name] = pulse
            if set(module['pulses'].values())==set(["high"]):
                return 'low', module['next']
        return 'high', module['next']

def process_round(modules):
    n = [0,0] #n_low, n_high
    cur_module_name = 'button'
    res_pulse, res_dst = module_process(None, modules[cur_module_name], 'low')
    todolist = [(cur_module_name, res_pulse, dst) for dst in res_dst]
    while (todolist):
        (from_module_name, pulse, dst) = todolist.pop(0)
        #print(from_module_name , '-', pulse, '-> ', dst)
        if pulse=='low': n[0]+=1
        else: n[1]+=1
        if dst not in modules:
            continue
        res_pulse, res_dst = module_process(from_module_name, modules[dst], pulse)
        if res_pulse is not None:
            todolist.extend([(dst, res_pulse, next_dst) for next_dst in res_dst])
    return n

In [3]:
import copy

modules = construct_modules("data/input20.txt")
modules = parse_conjunctions_pulses(modules)
n_button = 1000
res = 0
nl, nh = 0,0
for i in range(n_button):
    n = process_round(modules)
    nl += n[0]
    nh += n[1]
nl*nh

839775244

#### Part 2

In [4]:
def process_round_rx(tracking_name, target_module_name, modules):
    cur_module_name = 'button'
    res_pulse, res_dst = module_process(None, modules[cur_module_name], 'low')
    todolist = [(cur_module_name, res_pulse, dst) for dst in res_dst]
    n_high=0
    while (todolist):
        (from_module_name, pulse, dst) = todolist.pop(0)
        #print(from_module_name , '-', pulse, '-> ', dst)
        if dst not in modules:
            continue
        if target_module_name==dst and from_module_name==tracking_name:
            #print(from_module_name, pulse, dst)
            if pulse == 'high':
                n_high += 1
        res_pulse, res_dst = module_process(from_module_name, modules[dst], pulse)
        if res_pulse is not None:
            todolist.extend([(dst, res_pulse, next_dst) for next_dst in res_dst])
    return n_high>0

modules = construct_modules("data/input20.txt")
modules = parse_conjunctions_pulses(modules)
target_module_name = None
for key, value in modules.items():
    if 'rx' in value['next']:
        target_module_name = key
        break
target_module_tracking_names = []
for key, value in modules.items():
    if target_module_name in value['next']:
        target_module_tracking_names.append(key)
n_buttons = []
for tracking_name in target_module_tracking_names:
    input_modules = copy.deepcopy(modules)
    n_button = 0
    while (True):
        n_button += 1
        res = process_round_rx(tracking_name, target_module_name, input_modules)
        if res == True:
            n_buttons.append(n_button)
            break

In [5]:
import math

math.lcm(*n_buttons)

207787533680413